In [39]:
import os
from dotenv import load_dotenv
from google import genai
import openpyxl
import sys


In [40]:
file_name = "students.xlsx"
all_student_data = []

In [41]:
try:
    with open(file_name, 'rb') as f:
        workbook = openpyxl.load_workbook(f)

    sheet = workbook.active
    print(f"✅ Successfully loaded '{file_name}'.")

    all_student_data = list(sheet.iter_rows(values_only=True))

except FileNotFoundError:
    print("❌ Error 404! File Not Found.")
    sys.exit(1)
except Exception as e:
    print(f"❌ An error occurred while processing the Excel file: {e}")
    sys.exit(1)

✅ Successfully loaded 'students.xlsx'.


In [42]:
data_context_string = "\n".join([", ".join(map(str, row)) for row in all_student_data])

In [43]:
load_dotenv(override=True)
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    print("❌ API Key Error: GEMINI_API_KEY not found in .env file.")
    sys.exit(1)

client = genai.Client(api_key=GEMINI_API_KEY)

In [44]:
query = input("Enter your Student ID or Phone Number to get your information: ")

prompt = f"""
You are an intelligent data retrieval assistant.
Based *only* on the student data provided below, find the information for a student matching the given ID, Name, or Phone Number.
The data is structured as comma-separated values (CSV) where the first row is the header.
If there are multiple matches, return all matching rows. If no match is found, return the exact phrase: 'No data found'.
Write the output in a clear, readable format.
like this:
Name: John Doe
ID: 123456
Course Enrolled: Computer Science
Days: Saturday and Sunday
Time: 10:00 AM - 12:00 PM
Room No: Lab 1

--- STUDENT DATA ---
{data_context_string}
--- END OF DATA ---

Query: {query}
"""

In [45]:
try:
    response = client.models.generate_content(
        model="gemini-2.5-flash", contents=prompt
    )
    print("\n--- Retrieval Result ---")
    print(response.text)
    print("----------------------")

except Exception as e:
    print(f"❌ Gemini API Error: {e}")


--- Retrieval Result ---
Name: Hafiz M Imran
ID: 509154
Course Enrolled: Graphics Design Essentials
Days: Sat–Sun
Time: 07:00 PM – 09:00 PM
Room No: Lab 1
----------------------


In [46]:
#EXPLAINED STEPS:
# --- 1. Load Excel Data ---
 # Initialize a list to hold all data

# --- 2. Prepare Data for Prompt ---
# Convert the list of tuples into a clean, readable string (like a CSV)
# We join the header and data rows with newlines to form the data context

# --- 3. Setup Gemini Client ---

# --- 4. Prepare and Send Prompt ---